In [1]:
!python3 -m pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


In [2]:
!pip3 install arxiv==2.1.0
!pip3 install python-dotenv tiktoken
# !pip install openai==0.27.8
# !pip install openai==1.3.4
# !pip install openai==1.30.0
!pip install openai==1.45.0

  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=654c7a1e4c87890c5ce2ba169fa6feb4bb25a6df5be99e40c2a8fb27642ca1f3
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.2 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 1.9 MB/s eta 0:00:00-:--:--
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: openai
    Found existing installation: openai 1.30.0
    Uninstalling openai-1.30.0:
      Successfully uninst

In [3]:
import datetime as dt
import arxiv
import openai
import yaml
import json

In [4]:
import openai
from openai import OpenAI

In [5]:
from dotenv import load_dotenv
import os

In [6]:
load_dotenv()

True

In [7]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
SYSTEM = """
### 指示 ###
論文の内容を理解した上で，重要なポイントを箇条書きで3点書いてください。

### 箇条書きの制約 ###
- 最大3個
- 日本語
- 箇条書き1個を50文字以内

### 対象とする論文の内容 ###
{text}

### 出力形式 ###
タイトル(和名)

- 箇条書き1
- 箇条書き2
- 箇条書き3
"""

In [9]:
# arXivの更新頻度を加味して、5日前の論文を検索
N_DAYS = 5
# N_DAYS = 360

# MAX_RESULT = 24  # 取得する論文数の上限
# MAX_RESULT = 32  # 取得する論文数の上限

MAX_RESULT = 64  # 取得する論文数の上限


# MODEL_NAME = "gpt-3.5-turbo-0613"
# MODEL_NAME = "gpt-3.5-turbo-1106"
# MODEL_NAME = "gpt-3.5-turbo-0125"
MODEL_NAME = "gpt-4o-mini"

TEMPERATURE = 0.7
# OpenAIクライアントの初期化
client = OpenAI()

# テンプレートを用意
QUERY_TEMPLATE = '%28 ti:%22{}%22 OR abs:%22{}%22 %29 AND submittedDate: [{} TO {}]'

# 検索を行い、結果を取得する関数
def search_arxiv(keyword):
    # Construct the default API client.
    client = arxiv.Client()
    # 2日前からN_DAYS前までの論文を検索
    today = dt.datetime.today() - dt.timedelta(days=2)
    # today = dt.datetime.today()
    
    base_date = today - dt.timedelta(days=N_DAYS)
    query = QUERY_TEMPLATE.format(keyword, keyword, base_date.strftime("%Y%m%d%H%M%S"), today.strftime("%Y%m%d%H%M%S"))

    search = arxiv.Search(
        query=query,
        max_results=MAX_RESULT,
        sort_by=arxiv.SortCriterion.SubmittedDate,
        sort_order=arxiv.SortOrder.Descending,
    )

    # results = []
    # for result in search.results():
    #     # カテゴリーチェック
    #     # if not set(result.categories) & CATEGORIES:
    #     #     continue
    #     # 要約内でのキーワードの存在チェック
    #     if keyword.lower() in result.summary.lower():
    #         results.append(result)
    results = client.results(search)
    return results

# 論文の要約を取得する関数
def get_summary(result):
    text = f"title: {result.title}\nbody: {result.summary}"

    messages = [
        {"role" : "system", "content" : SYSTEM},
        {"role": "user", "content": text}
    ]
    
    # response = openai.ChatCompletion.create(
    #     model=MODEL_NAME,
    #     messages=messages,
    #     temperature=TEMPERATURE,
    # )
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=TEMPERATURE,
    )
    return response.choices[0].message.content
    # return response["choices"][0]["message"]["content"]

In [10]:
def get_summary(result):
    model_name=MODEL_NAME
    text=result.summary
    # 概要と提案手法名を生成する
    prompt = [{'role': 'system', 'content': "Please compile the following text into a research report so that it can be fully understood without any omissions. If a method is proposed, extract its name."}]
    prompt.append({"role" : "system", "content" : 'Please generate the research report to include the following contents.\n"CHALLENGE" The current situation faced by the researcher; it will normally include a Problem Statement, the Motivation, a Hypothesis and/or a Goal.\n"APPROACH" How they intend to carry out the investigation, comments on a theoretical model or framework.\n"OUTCOME" Overall conclusion that should reject or support the research hypothesis.'})    
    prompt.append({"role" : "system", "content" : 'The text should be explained using language and mathematical formulas that a first-year college student can understand.'})
    
    prompt.append({"role" : "system", "content" : 'Please create a research report by referencing the example research report below, structuring it around three distinct aspects in order: "Challenge," "Approach," and "Outcome."\n"Challenge" Existing statistical phrasal orhierarchical machine translation system relies on a large set of translation rules which results in engineering challenges. \n"Approach": The proposed method consistently outperforms existing methods in BLEU on various low-resource language translation tasks with less training data.\n"Outcome" They propose to use factorized grammar from the field of linguistics as more general translation rules from XTAG English Grammar.'})
    
    prompt.append({"role": "system", "content": "Results must be in Japanese."})
    prompt.append({"role" : "system", "content" : "Outputs should be generated in step by step."})
    prompt.append({"role": "system", "content": "Think the option as hypothesis. Whether it entails with those premises?"})
    prompt.append({"role" : "system", "content" : "Please explain the research report in a way that is easy to understand for high school students, without making it complicated. It's okay if the explanation becomes lengthy."})
    
    prompt.append({"role": "system", "content": "Please format the output in Markdown."})
    prompt.append({"role": "system", "content": "Results must be in Japanese."})
    
    prompt.append({"role": "user", "content": 'Based on the input text, generate a JSON containing two different pieces of information. First, use "extract_summary" as the schema to create a section that keys in the result of summarizing the content of the text accurately and completely. Next, use "method_name" as the schema to create a section that keys in the name of the method proposed or used within the text. Combine these pieces of information to output as a single JSON object. This JSON will be formatted as {"extract_summary": [the result of summarizing the text content], "method_name": [the name of the method proposed or used within the text]}.'})
    
    prompt.append({"role": "user", "content": f"Input text: {text}"})
    prompt.append({"role": "user", "content": f"Include:\n- Overview\n- Novelty\n- Methodology\n- Results"})
    prompt.append({"role": "system", "content": "Results must be in Japanese."})
    """
    システム
    あなたは以下の text を過不足なく理解できるように調査報告書としてまとめ、提案されている手法がある場合はその名称を抽出してください。
    文章は大学一年生が理解できる程度の言葉と数式を用いて説明してください。
    調査報告書は以下の内容を含む形で生成してください\n
    • CHALLENGE: The current situation faced by the researcher; it will normally include a Problem Statement, the Motivation, a Hypothesis and/or a Goal.\n
    • APPROACH: How they intend to carry out the investigation, comments on a theoretical model or framework.\n
    • OUTCOME: Overall conclusion that should reject or support the research hypothesis.

    調査報告書は以下の例を参考に「課題」「アプローチ」「結果」の3つの異なる側面を順番に作成してください。概要: [ACLSum: A New Dataset for Aspect-based Summarization of Scientific Publications](https://arxiv.org/abs/2403.05303v1)のデータセット例を一つ
    出力は Markdown 形式にしてください。
    
    結果は日本語でなければならない。
    user
    入力されたテキストに基づき、二つの異なる情報を含むJSONを生成します。最初に、"extract_summary"をスキーマとして使用し、テキストの内容を過不足なく概要としてまとめた結果をキーとする部分を生成します。次に、"method_name"をスキーマとして使用し、テキスト内で提案されたり使用されている手法名をキーとする部分を生成します。これらの情報を組み合わせて、一つのJSONオブジェクトとして出力します。このJSONは、{"extract_summary": [テキストの内容を概要としてまとめた結果], "method_name": [テキスト内で提案されたり使用されている手法名]}の形式で表されます。
    
    入力されたテキスト: {text}
    """
    
    # 概要と提案手法名抽出用のプロンプトテンプレートを作成
    response = client.chat.completions.create(
        model=model_name, # model = "deployment_name".
        messages=prompt,
        # response_format={ "type": "json_object" },
        temperature=TEMPERATURE,
    )
    summary_method_name_str = response.choices[0].message.content

    return summary_method_name_str
    # print(summary_method_name_str)
    
    # # JSON形式の文字列を辞書に変換
    # summary_method_name = json.loads(summary_method_name_str)
    
    # 出力と新しいメッセージをステートに反映
    # return summary_method_name["extract_summary"]

In [11]:
# CATEGORIES = {
#     "cs.AI",  # 例: コンピュータサイエンスのAI分野
#     # "stat.ML",  # 例: 統計学の機械学習分野
#     # 必要に応じて他のカテゴリーを追加
# }

keyword = "LLM"
# keyword = "LLM zero"
# keyword = "LLM prompt"
# keyword = "prompt optimization"
# keyword = "Prompt Generation"
# keyword = "LLM code"

results = search_arxiv(keyword)

for result in results:
    summary = get_summary(result)
    # summaryの中の"."を改行"\n"に置き換える
    formatted_summary = summary.replace('。', '。\n')

    print(f"title: {result.title}")
    print(f"published: {result.published}")
    print(f"abstruct: {result.summary}")
    print(f"PDFリンク: {result.pdf_url}")
    # print(f"summary: {summary}")
    print(f"summary:\n{formatted_summary}")
    print("-" * 50)


title: Controlling Language and Diffusion Models by Transporting Activations
published: 2024-10-30 14:21:33+00:00
abstruct: The increasing capabilities of large generative models and their ever more
widespread deployment have raised concerns about their reliability, safety, and
potential misuse. To address these issues, recent works have proposed to
control model generation by steering model activations in order to effectively
induce or prevent the emergence of concepts or behaviors in the generated
output. In this paper we introduce Activation Transport (AcT), a general
framework to steer activations guided by optimal transport theory that
generalizes many previous activation-steering works. AcT is modality-agnostic
and provides fine-grained control over the model behavior with negligible
computational overhead, while minimally impacting model abilities. We
experimentally show the effectiveness and versatility of our approach by
addressing key challenges in large language models (LLMs

In [12]:
# keyword = "LLM Agent"
# # keyword = "Document LLM"

# results = search_arxiv(keyword)

# for result in results:
#     summary = get_summary(result)
#     print(f"title: {result.title}")
#     print(f"published: {result.published}")
#     print(f"abstruct: {result.summary}")
#     print(f"PDFリンク: {result.pdf_url}")
#     print(f"summary: {summary}")
#     print("-" * 50)


In [13]:
# keyword = "RAG"

# results = search_arxiv(keyword)

# for result in results:
#     summary = get_summary(result)
#     print(f"title: {result.title}")
#     print(f"published: {result.published}")
#     print(f"abstruct: {result.summary}")
#     print(f"PDFリンク: {result.pdf_url}")
#     print(f"summary: {summary}")
#     print("-" * 50)


In [14]:
# keyword = "LLM write"
# # keyword = "Document LLM"

# results = search_arxiv(keyword)

# for result in results:
#     summary = get_summary(result)
#     print(f"title: {result.title}")
#     print(f"published: {result.published}")
#     print(f"abstruct: {result.summary}")
#     print(f"PDFリンク: {result.pdf_url}")
#     print(f"summary: {summary}")
#     print("-" * 50)


In [15]:
# keyword = "text writing"
# # keyword = "Document LLM"

# results = search_arxiv(keyword)

# for result in results:
#     summary = get_summary(result)
#     print(f"title: {result.title}")
#     print(f"published: {result.published}")
#     print(f"abstruct: {result.summary}")
#     print(f"PDFリンク: {result.pdf_url}")
#     print(f"summary: {summary}")
#     print("-" * 50)


In [16]:
# keyword = "Knowledge graph"
# # keyword = "Document LLM"

# results = search_arxiv(keyword)

# for result in results:
#     summary = get_summary(result)
#     print(f"title: {result.title}")
#     print(f"published: {result.published}")
#     print(f"abstruct: {result.summary}")
#     print(f"PDFリンク: {result.pdf_url}")
#     print(f"summary: {summary}")
#     print("-" * 50)
